In [1]:
import matplotlib.pyplot as plt
import pandas as pd 
import spotipy 
import librosa
import librosa.display
import numpy as np
import matplotlib as mpl
import urllib.request
import seaborn as sns
import tqdm
import scipy
import joblib
import os.path
from pathlib import Path
from IPython.display import Audio, Markdown, Image
from spotipy.oauth2 import SpotifyClientCredentials

# Números y Datos
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import interp1d

# Análisis de sonido
import librosa
import librosa.display
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 

# Machine learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from sklearn.preprocessing import quantile_transform

# Styles
sns.set_context('poster')
sns.set_style('darkgrid')

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [2]:
# Extraemos los dataframes de nuestros files
df_tracks = pd.read_pickle('../sources/tracks.pickle')
df_af = pd.read_pickle('../sources/audio_features.pickle')
df_aa = pd.read_pickle('../sources/audio_analysis.pickle') \
    .set_index(['id','start']) \
    .sort_values(by=['id', 'start'], ascending=True, na_position='first')

In [3]:
# Generamos un dataframe combinando la data de tracks y de audio features
df_merged = df_tracks.merge(
    df_af, 
    on='id', 
    how='left'
)
df_merged = df_merged.drop(
    [
     'type_x',
     'type_y',
     'uri_x',
     'uri_y',
     'track_href',
     'analysis_url',
     'href',
     'preview_url',
     'external_ids',
     'duration_ms_y'
    ], 
    1
)

df_merged.rename(columns = {"duration_ms_x": "duration_ms"}, inplace = True)

In [4]:
# Extraemos para cada registro la variable release_date y la asignamos a una nueva columna en nuestro dataframe.

# Función que generamos para formatear la fecha
def date_formator(date):
    if '-' in date:
        year = pd.to_datetime(date, format = '%Y-%m-%d').year
        return int(year)
    elif int(date)>0:
        year = pd.to_datetime(date, format = '%Y').year
        return int(year)
    return None


df_merged['release_date'] = df_merged['album'].map(lambda x: x['release_date'])
df_merged['release_date'] = df_merged['release_date'].apply(date_formator)
df_merged['release_date'].head()

id
4d3XHYFFuYYzxWr2cJ6yQl    1991.0
3X3p3u03P8eFL8WTH0oaaU    1958.0
4KlXeaheot0OI9PoOWspvZ    2014.0
3j76McoUI18gDoqxYE14Bu    2014.0
4zxd4tiXPlWMqoJltbVTbE    1959.0
Name: release_date, dtype: float64

In [5]:
# Obtengo ids de album
df_merged['album_id'] = df_merged['album'].map(lambda x: x['id'])

# Agrego artistas y género
df_merged[["artists","genre"]] =  df_merged.loc[:,["artists","genre"]] 

In [6]:
# Vemos cuales son nuestras columnas
df_merged.columns.values

array(['album', 'artists', 'available_markets', 'disc_number',
       'duration_ms', 'explicit', 'external_urls', 'is_local', 'name',
       'popularity', 'track_number', 'genre', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'release_date', 'album_id'], dtype=object)

In [7]:
df_merged.sample(3).T

id,06dpGiZGqkwqGAp3hcZsau,5g7rJvWYVrloJZwKiShqlS,3r94oMT1M8jF89PSw8qU00
album,"{'album_type': 'ALBUM', 'artists': [{'external...","{'album_type': 'ALBUM', 'artists': [{'external...","{'album_type': 'ALBUM', 'artists': [{'external..."
artists,Solar Fields,Of Monsters and Men,The Souljazz Orchestra
available_markets,[],"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",[]
disc_number,1,1,1
duration_ms,450535,278373,248000
explicit,False,False,False
external_urls,{'spotify': 'https://open.spotify.com/track/06...,{'spotify': 'https://open.spotify.com/track/5g...,{'spotify': 'https://open.spotify.com/track/3r...
is_local,False,False,False
name,Feelings - Album edit,Dirty Paws,Kossa Kossa
popularity,0,69,0


Desdoblamos los features de pitches y timbre en distintas columnas

In [16]:
df_m2 = df_aa

pitches_nombre = ['p00_C','p01_C#','p02_D','p03_D#','p04_E','p05_F',
                 'p06_F#','p07_G','p08_G#','p09_A','p10_A#','p11_B']
df1 = pd.DataFrame(df_m2['pitches'].tolist(), columns=pitches_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1)

timbre_nombre = ['t00','t01','t02','t03','t04','t05',
                 't06','t07','t08','t09','t10','t11']
df1 = pd.DataFrame(df_m2['timbre'].tolist(), columns=timbre_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1).drop(['pitches','timbre'],1)

df1 = None
df_m2.head(3).T

id                00At7PWydsvg7g5xgaYan9                      
start                            0.00000    0.24381    0.41873
duration                         0.24381    0.17492    0.17329
confidence                       0.00000    1.00000    1.00000
loudness_start                 -60.00000  -60.00000  -34.27900
loudness_max_time                0.00000    0.01436    0.01554
loudness_max                   -60.00000  -12.45700  -12.72800
loudness_end                     0.00000    0.00000    0.00000
p00_C                            1.00000    0.12100    0.03200
p01_C#                           1.00000    0.99600    1.00000
p02_D                            1.00000    0.08000    0.06000
p03_D#                           1.00000    0.10400    0.05900
p04_E                            1.00000    1.00000    0.36800
p05_F                            1.00000    0.22300    0.08400
p06_F#                           1.00000    0.04300    0.03600
p07_G                            1.00000    0.04700    0.04100
p08_G#                           1.00000    0.17600    0.10600
p09_A                            1.00000    0.42200    0.38000
p10_A#                           1.00000    0.12900    0.11200
p11_B                            1.00000    0.06200    0.08700
t00                              0.00000   37.64100   38.17100
t01                            171.13000  -77.64300  -17.74100
t02                              9.46900  -30.61300  -22.79800
t03                            -28.48000  137.46200  149.22300
t04                             57.49100   36.96000   31.83300
t05                            -50.06700  -51.02400  -53.44600
t06                             14.83300  -18.74500  -36.48500
t07                              5.35900  -28.82200  -39.47900
t08                            -27.22800   -7.17000  -25.79500
t09                              0.97300   -1.99400   -1.90300
t10                            -10.64000  -15.69500  -21.90800
t11                             -7.22800    9.41100   15.93400

Extraemos los datos de MEDIANA + IQR (RANGO INTERCUARTIL)

In [17]:
grouped = df_m2.drop(['duration'],1).groupby(level='id')
medianas = grouped.quantile(.5)
iqr = grouped.quantile(.75)-grouped.quantile(.25)
medias = grouped.mean()
desvios = grouped.std()

In [18]:
df_pitches_timbres_medias_std = pd.merge(medias, desvios, on='id', suffixes=('_media', '_std'))
df_pitches_timbres_medianas_iqr = pd.merge(medianas, iqr, on='id', suffixes=('_mediana', '_iqr'))

Y lo unimos al otro dataset

In [34]:
df_merged2 = df_merged.select_dtypes(['number']).drop(['disc_number','track_number','release_date'],1)

df_todo = df_merged2.merge(
    df_pitches_timbres_medianas_iqr, 
    on='id', 
    how='inner'
)
df_todo['genre']=df_merged['genre']

id,4d3XHYFFuYYzxWr2cJ6yQl,3X3p3u03P8eFL8WTH0oaaU,4KlXeaheot0OI9PoOWspvZ
duration_ms,349600,344066,348746
popularity,49,47,13
danceability,0.487,0.615,0.792
energy,0.145,0.204,0.301
key,5,7,9
loudness,-18.67,-16.075,-9.925
mode,1,1,0
speechiness,0.0522,0.0439,0.0409
acousticness,0.876,0.872,0.546
instrumentalness,0.822,0.382,0.821


Y ahora hacemos lo mismo para la derivada de las funciones

In [43]:
dff = df_m2
shifted = dff.groupby(level="id").shift(-1)
dif = dff.drop(['duration'],1).subtract(shifted.drop(['duration'],1), fill_value=0)
diferencias = dif.divide(dff['duration'], axis="index")
diferencias.head(10)

confidence  loudness_start  loudness_max_time  \
id                     start                                                    
00At7PWydsvg7g5xgaYan9 0.00000   -4.101554        0.000000          -0.058898   
                       0.24381    0.000000     -147.044363          -0.006746   
                       0.41873    0.000000      -10.416066           0.028565   
                       0.59202    0.000000        1.798167          -0.011239   
                       0.76553    0.000000      -20.086516           0.014287   
                       0.94122    0.000000       -5.067100          -0.025155   
                       1.10739    0.000000       18.555168           0.006921   
                       1.28222    0.000000      -21.987248          -0.072723   
                       1.44063    0.145529       -4.096373           0.063548   
                       1.62617   -0.155521       20.782213           0.006336   

                                loudness_max  loudness_end     p00_C  \
id                     start                                           
00At7PWydsvg7g5xgaYan9 0.00000   -195.000205           0.0  3.605266   
                       0.24381      1.549280           0.0  0.508804   
                       0.41873    -10.064054           0.0 -1.465751   
                       0.59202     -8.668088           0.0  1.279465   
                       0.76553     29.335762           0.0 -0.506574   
                       0.94122    -24.601312           0.0 -0.337004   
                       1.10739    -15.094663           0.0  0.875136   
                       1.28222      2.518780           0.0 -0.018938   
                       1.44063     30.059829           0.0 -1.870317   
                       1.62617    -19.382524           0.0  1.774091   

                                  p01_C#     p02_D    p03_D#     p04_E  ...  \
id                     start                                            ...   
00At7PWydsvg7g5xgaYan9 0.00000  0.016406  3.773430  3.674993  0.000000  ...   
                       0.24381 -0.022868  0.114338  0.257260  3.613080  ...   
                       0.41873  0.000000 -1.021409 -1.258007 -1.881240  ...   
                       0.59202  0.703130  1.204542  0.852977 -1.763587  ...   
                       0.76553 -0.068302 -0.239057  0.409813  2.942683  ...   
                       0.94122 -0.661973 -0.306915 -0.355058  0.096287  ...   
                       1.10739  0.000000  0.343191  0.011440 -0.228794  ...   
                       1.28222  0.000000  0.126255  0.113629 -0.782779  ...   
                       1.44063  0.000000 -1.352881 -1.573869 -1.040263  ...   
                       1.62617  0.000000  1.123207  1.866252  3.196820  ...   

                                       t02         t03         t04  \
id                     start                                         
00At7PWydsvg7g5xgaYan9 0.00000  164.398507 -680.620155   84.209015   
                       0.24381  -44.677567  -67.236451   29.310542   
                       0.41873 -184.413411  -85.769519   -9.475446   
                       0.59202  345.023342  196.138551  219.422512   
                       0.76553 -110.490068  293.078718 -125.129489   
                       0.94122 -216.501173 -442.805561 -293.235843   
                       1.10739  233.220843  127.460962  155.711262   
                       1.28222  -98.813206   14.279402 -133.937251   
                       1.44063   -3.864604   73.804776  104.263461   
                       1.62617 -324.935200 -381.850124 -240.176257   

                                       t05         t06         t07  \
id                     start                                         
00At7PWydsvg7g5xgaYan9 0.00000    3.925188  137.721997  140.195234   
                       0.24381   13.846330  101.417791   60.924994   
                       0.41873  174.124300   83.969069  -92.180737   
                       0.59202   -4.933433 -106.276295

In [44]:
grouped_dif = diferencias.groupby(level='id')
medianas = grouped_dif.quantile(.5)
iqr = grouped_dif.quantile(.75)-grouped.quantile(.25)
medias = grouped_dif.mean()
desvios = grouped_dif.std()

In [45]:
dif_pitches_timbres_medias_std = pd.merge(medias, desvios, on='id', suffixes=('_dif_media', '_dif_std'))
dif_pitches_timbres_medianas_iqr = pd.merge(medianas, iqr, on='id', suffixes=('_dif_mediana', '_dif_iqr'))
len(dif_pitches_timbres_medias_std)

4177

Y lo unimos al otro dataset

In [47]:
df_merged2 = df_todo

df_todo = df_merged2.merge(
    dif_pitches_timbres_medianas_iqr, 
    on='id', 
    how='inner'
)
df_todo['genre']=df_merged['genre']

In [48]:
df_todo.to_pickle("../sources/df_todo_0_sin_escalar.pickle")

df_todo.head(3).describe().T.head(30)

,count,mean,std,min,25%,50%,75%,max
duration_ms,3.0,347470.666667,2979.286044,344066.00000,346406.000000,348746.000000,349173.000000,349600.00000
popularity,3.0,36.333333,20.231988,13.00000,30.000000,47.000000,48.000000,49.00000
danceability,3.0,0.631333,0.153155,0.48700,0.551000,0.615000,0.703500,0.79200
energy,3.0,0.216667,0.078768,0.14500,0.174500,0.204000,0.252500,0.30100
key,3.0,7.000000,2.000000,5.00000,6.000000,7.000000,8.000000,9.00000
loudness,3.0,-14.890000,4.491317,-18.67000,-17.372500,-16.075000,-13.000000,-9.92500
mode,3.0,0.666667,0.577350,0.00000,0.500000,1.000000,1.000000,1.00000
speechiness,3.0,0.045667,0.005853,0.04090,0.042400,0.043900,0.048050,0.05220
acousticness,3.0,0.764667,0.189381,0.54600,0.709000,0.872000,0.874000,0.87600
instrumentalness,3.0,0.675000,0.253746,0.38200,0.601500,0.821000,0.821500,0.82200


In [49]:

df_todo.head(3).describe().T.tail(30)

,count,mean,std,min,25%,50%,75%,max
t11_dif_mediana,3.0,-0.360272,1.464898,-1.772372,-1.116536,-0.460700,0.345778,1.152256
confidence_dif_iqr,3.0,0.380887,0.250064,0.143100,0.250509,0.357917,0.499780,0.641644
loudness_start_dif_iqr,3.0,43.234278,4.102798,39.342477,41.091567,42.840657,45.180179,47.519701
loudness_max_time_dif_iqr,3.0,0.070554,0.040899,0.025637,0.053008,0.080380,0.093013,0.105646
loudness_max_dif_iqr,3.0,33.700533,3.920333,31.071203,31.447584,31.823965,35.015199,38.206433
loudness_end_dif_iqr,3.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
p00_C_dif_iqr,3.0,0.569735,0.208562,0.416047,0.451028,0.486009,0.646579,0.807150
p01_C#_dif_iqr,3.0,0.412512,0.302802,0.155533,0.245603,0.335673,0.541002,0.746332
p02_D_dif_iqr,3.0,0.399850,0.165192,0.209119,0.351110,0.493101,0.495216,0.497331
p03_D#_dif_iqr,3.0,0.300797,0.098381,0.212475,0.247780,0.283085,0.344958,0.406832


Y LO NORMALIZAMOS (al menos hasta donde puede hacerse de una manera sencilla)

Primero con MinMaxScaler

Y despues con Normal Quantiles

In [50]:
df_para_escalar = df_todo.select_dtypes(['number'])

scaler = MinMaxScaler(feature_range=(0,1))
df_todo_MinMaxScaled = pd.DataFrame(
    scaler.fit_transform(df_para_escalar),
    columns=df_para_escalar.columns,index=df_para_escalar.index
)
df_todo_MinMaxScaled['genre']=df_todo['genre']

df_todo_MinMaxScaled.to_pickle("../sources/df_todo_1_MinMaxScaled.pickle")

df_todo_MinMaxScaled.describe().T.head(30)

,count,mean,std,min,25%,50%,75%,max
duration_ms,4082.0,0.071738,0.044974,0.0,0.045713,0.062317,0.085867,1.0
popularity,4082.0,0.220605,0.240983,0.0,0.000000,0.162791,0.383721,1.0
danceability,4082.0,0.467091,0.196402,0.0,0.312896,0.476216,0.613901,1.0
energy,4082.0,0.552175,0.340881,0.0,0.217982,0.578913,0.897736,1.0
key,4082.0,0.471182,0.321789,0.0,0.181818,0.454545,0.727273,1.0
loudness,4082.0,0.736707,0.164114,0.0,0.641785,0.784772,0.864777,1.0
mode,4082.0,0.644047,0.478860,0.0,0.000000,1.000000,1.000000,1.0
speechiness,4082.0,0.077912,0.066009,0.0,0.044224,0.054492,0.081884,1.0
acousticness,4082.0,0.416802,0.420862,0.0,0.001923,0.218876,0.908635,1.0
instrumentalness,4082.0,0.387383,0.394439,0.0,0.000741,0.211616,0.841414,1.0


In [51]:
df_para_escalar = df_todo.select_dtypes(['number'])

quantil=100

df_todo_quantiles_normal = pd.DataFrame(
    quantile_transform(
        df_para_escalar,
        n_quantiles = quantil,
        output_distribution = 'normal', 
        random_state = 0, 
        copy = True
    ), columns=df_para_escalar.columns,index=df_para_escalar.index
)
df_todo_quantiles_normal['genre']=df_todo['genre']

df_todo_quantiles_normal.to_pickle("../sources/df_todo_2_NormalScaled.pickle")

df_todo_quantiles_normal.describe().T.head(30)

,count,mean,std,min,25%,50%,75%,max
duration_ms,4082.0,-0.001851,9.926864e-01,-5.199338,-0.678479,0.000789,0.672820,5.199338
popularity,4082.0,-1.769321,2.929961e+00,-5.199338,-5.199338,0.012660,0.666564,5.199338
danceability,4082.0,0.000725,9.962546e-01,-5.199338,-0.673879,-0.000275,0.671317,5.199338
energy,4082.0,-0.000224,1.003355e+00,-5.199338,-0.674490,-0.001582,0.675625,5.199338
key,4082.0,-0.173515,2.295738e+00,-5.199338,-0.619855,-0.050661,0.635270,5.199338
loudness,4082.0,-0.001218,9.945427e-01,-5.199338,-0.674745,0.000878,0.676413,5.199338
mode,4082.0,1.497898,4.979507e+00,-5.199338,-5.199338,5.199338,5.199338,5.199338
speechiness,4082.0,0.000037,9.914065e-01,-5.199338,-0.677141,0.000342,0.675842,5.199338
acousticness,4082.0,0.004234,1.022113e+00,-5.199338,-0.673852,-0.002030,0.678468,5.199338
instrumentalness,4082.0,-0.273228,1.682350e+00,-5.199338,-0.673508,-0.000762,0.675625,5.199338


In [52]:
df_para_escalar = df_todo.select_dtypes(['number'])

quantil=100

df_todo_quantiles_uniform = pd.DataFrame(
    quantile_transform(
        df_para_escalar,
        n_quantiles = quantil,
        output_distribution = 'uniform', 
        random_state = 0, 
        copy = True
    ), columns=df_para_escalar.columns,index=df_para_escalar.index
)
df_todo_quantiles_uniform['genre']=df_todo['genre']

df_todo_quantiles_uniform.to_pickle("../sources/df_todo_3_UniformScaled.pickle")

df_todo_quantiles_uniform.describe().T.head(30)

,count,mean,std,min,25%,50%,75%,max
duration_ms,4082.0,0.499949,0.288585,0.0,0.248734,0.500315,0.749469,1.0
popularity,4082.0,0.413886,0.371783,0.0,0.000000,0.505051,0.747475,1.0
danceability,4082.0,0.499944,0.288671,0.0,0.250194,0.499890,0.748990,1.0
energy,4082.0,0.500007,0.288791,0.0,0.250000,0.499369,0.750361,1.0
key,4082.0,0.495759,0.301745,0.0,0.267677,0.479798,0.737374,1.0
loudness,4082.0,0.499943,0.288647,0.0,0.249919,0.500350,0.750611,1.0
mode,4082.0,0.644047,0.478860,0.0,0.000000,1.000000,1.000000,1.0
speechiness,4082.0,0.500019,0.288623,0.0,0.249158,0.500137,0.750430,1.0
acousticness,4082.0,0.500171,0.289000,0.0,0.250203,0.499190,0.751263,1.0
instrumentalness,4082.0,0.496636,0.294146,0.0,0.250312,0.499696,0.750361,1.0
